# Práctica: MANOVA

Eres parte de un equipo de investigación que evalúa la rehabilitación neuropsicológica de pacientes que entraron en estado de coma. Actualmente, se te ha pedido que indagues sobre características sociodemográficas que puedan potencialmente influir en la rehabilitación de estos pacientes. Para ello se te brindan los siguientes datos:

In [ ]:
!pip install pingouin

In [1]:
import pandas as pd
from sklearn.preprocessing import PowerTransformer

# Load and prepare data
df = pd.read_csv(
    "https://socialsciences.mcmaster.ca/jfox/Books/Applied-Regression-3E/datasets/Wong.txt",
    sep=r"\s{1,}",
    engine="python",
)
df.dropna(inplace=True)
category = pd.cut(
    df.age, bins=[0, 17, 65, 99], labels=["minor", "adult", "elder"]
)
df.drop(columns=["id", "duration", "days", "age"], inplace=True)
wong_data = pd.get_dummies(df, drop_first=True)
wong_data.insert(3, "age_group", category)

# Enforce normality for teaching purposes
scaler = PowerTransformer()
scaled_data = scaler.fit_transform(wong_data[["piq", "viq"]])
wong_data["piq"] = scaled_data[:, 0]
wong_data["viq"] = scaled_data[:, 1]
wong_data

,piq,viq,sex_Male,age_group
0,0.018012,-0.377590,1,adult
1,0.534757,-1.328667,1,adult
2,0.534757,1.440707,1,adult
3,-2.075023,-1.671548,1,adult
4,-1.422233,-1.671548,1,adult
...,...,...,...,...
326,0.904403,-0.760845,1,adult
327,0.279997,-0.452864,1,adult
328,0.084214,0.612245,1,minor
329,0.904403,0.060611,1,minor


1. Evalúa el supuesto de normalidad univariante y multivariante de las variables dependientes. 

In [2]:
dep_vars = ["piq", "viq"]
ind_vars = ["sex_Male", "age_group"]

import pingouin as pg

pg.multivariate_normality(wong_data[dep_vars])

HZResults(hz=1.0706313169235724, pval=0.05330264969552362, normal=True)

In [3]:
pg.normality(wong_data[dep_vars])

,W,pval,normal
piq,0.996002,0.568481,True
viq,0.991983,0.070995,True


El supuesto de normalidad multivariante de las variables dependientes se cumple satisfactoriamente. También se cumple a nivel univariante. 

2. Evalúa el supuesto de homocedasticidad univariante y multivariante de las variables dependientes.

In [4]:
# Compute multivariate homocedasticity
multi_homoek = pd.DataFrame()
for var in ind_vars:
    frame = pg.box_m(data=wong_data, dvs=dep_vars, group=var)
    multi_homoek = multi_homoek.append(frame.rename(index={"box": var}))
multi_homoek

,Chi2,df,pval,equal_cov
sex_Male,5.513909,3.0,0.137809,True
age_group,5.672620,6.0,0.460844,True


In [5]:
from itertools import chain

lev = pd.DataFrame()
for ivar in ind_vars:
    for dvar in dep_vars:
        frame = pg.homoscedasticity(data=wong_data, dv=dvar, group=ivar)
        lev = lev.append(frame.rename(index={"levene": dvar}))
lev.insert(0, "X", list(chain(*[[i] * len(dep_vars) for i in ind_vars])))
lev

,X,W,pval,equal_var
piq,sex_Male,3.894847,0.049271,False
viq,sex_Male,0.516699,0.472764,True
piq,age_group,2.143946,0.118831,True
viq,age_group,1.386865,0.251318,True


Las pruebas de Box M indican que el supuesto de igualdad de matrices de covarianzas se cumple. Sin embargo, no encontramos homocedasticidad univariante cuando evaluamos la variable piq agrupada por el sexo de los participantes (variable sex_Male).

3. Verifica que las variables dependientes se encuentren correlacionadas, pero no presenten multicolinealidad extrema. 

In [6]:
wong_data[dep_vars].corr()

,piq,viq
piq,1.0000,0.6429
viq,0.6429,1.0000


In [7]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

statistic, p_value = calculate_bartlett_sphericity(wong_data[dep_vars])
print("Chi-squared: ", statistic)
print("p-value: ", p_value)

Chi-squared:  175.1813178438344
p-value:  5.465270203079977e-40


La prueba de esfericidad de Bartlett indica que existen correlaciones significativas entre las variables. Al evaluar los coeficientes, encontramos que no hay correlaciones por encima de 0.9, por lo que se descarta multicolinealidad extrema. 

4. Implementa un modelo MANOVA que satisfaga la ecuación $ Y_{piq} + Y_{viq} = X_{sex} + X_{age \ group} $. ¿Qué puedes concluir?

In [8]:
from statsmodels.multivariate.manova import MANOVA

maov = MANOVA.from_formula("piq + viq  ~ sex_Male * age_group", data=wong_data)
print(maov.mv_test())

                  Multivariate linear model
                                                             
-------------------------------------------------------------
       Intercept        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.9937 2.0000 324.0000  1.0205 0.3616
         Pillai's trace 0.0063 2.0000 324.0000  1.0205 0.3616
 Hotelling-Lawley trace 0.0063 2.0000 324.0000  1.0205 0.3616
    Roy's greatest root 0.0063 2.0000 324.0000  1.0205 0.3616
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
       age_group        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.9813 4.0000 648.0000  1.5383 0.1894
         Pillai's trace 0.0188 4.0000 650.0000  1.5421 0.1883
 Hotelling-Lawley trace 0.

Debido a que el modelo no cumple estrictamente con todos los supuestos, optamos por analizar el criterio de Pillai. De acuerdo a este criterio no existen efectos principales ni interacciones significativas en el modelo. 

5. Verifica si las diferencias significativas encontradas con tu modelo MANOVA se replican con cada variable dependiente por separado. ¿Qué puedes concluir?

In [9]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

for dep in dep_vars:
    model = smf.ols(formula=f"{dep} ~ sex_Male*age_group", data=wong_data)
    print("_" * 100)
    print(dep)
    print(sm.stats.anova_lm(model.fit()))

____________________________________________________________________________________________________
piq
                       df      sum_sq   mean_sq         F    PR(>F)
age_group             2.0    0.398722  0.199361  0.198371  0.820164
sex_Male              1.0    0.794283  0.794283  0.790340  0.374655
sex_Male:age_group    2.0    3.185660  1.592830  1.584923  0.206544
Residual            325.0  326.621335  1.004989       NaN       NaN
____________________________________________________________________________________________________
viq
                       df      sum_sq   mean_sq         F    PR(>F)
age_group             2.0    1.214393  0.607197  0.602760  0.547910
sex_Male              1.0    0.291551  0.291551  0.289420  0.590961
sex_Male:age_group    2.0    2.101741  1.050871  1.043192  0.353505
Residual            325.0  327.392315  1.007361       NaN       NaN


Cuando se evalúa cada variable dependiente por separado no se encuentran efectos principales ni interacciones significativas.  